In [34]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [35]:
csv_file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week3\project\Data\Literacy\literacy-rate-adults.csv'
df_lit_original= pd.read_csv(csv_file_path)
df_lit = df_lit_original.copy()
df_lit = df_lit[df_lit['Year'] >= 1991]
df_lit.columns

Index(['Entity', 'Code', 'Year',
       'Literacy rate, adult total (% of people ages 15 and above)'],
      dtype='object')

In [36]:
csv_file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week3\project\Data\Unemployment\unemployment-rate.csv'
df_un_original = pd.read_csv(csv_file_path)
df_un = df_un_original.copy()
df_un['Year'].unique()

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [41]:
merged_df = pd.merge(df_lit, df_un, on=['Entity', 'Year'], how='inner')
merged_df = merged_df.drop(columns=['Code_x', 'Code_y'] , errors='ignore')
merged_df.rename(columns={'Entity': 'country', 'Literacy rate, adult total (% of people ages 15 and above)' : 'literacy_rate_%','Unemployment, total (% of total labor force) (modeled ILO estimate)' : 'unemployment_rate_%', 'Year' : 'year'}, inplace=True)
grouped_df = merged_df.groupby(['country', 'year']).agg({'literacy_rate_%': 'first', 'unemployment_rate_%' : 'first'})
grouped_df.head(40)

literacy_rate_%  unemployment_rate_%
country     year                                      
Afghanistan 2011         31.44885               11.054
            2021         37.26604               13.283
Albania     2001         98.71298               18.575
            2008         95.93864               13.060
            2011         96.84530               13.480
            2012         97.24697               13.380
            2018         98.14115               12.300
            2021         98.45000               11.819
Algeria     2002         69.87350               25.900
            2006         72.64868               12.270
            2008         75.13605               11.330
            2018         81.40784               10.420
Angola      2001         67.40542                4.004
            2014         66.03011                7.372
            2021         72.28000                8.530
Argentina   1991         96.04072                5.440
            2001         97.19331               17.320
            2006         98.61080               10.080
            2009         98.98342                8.650
            2010         98.95497                7.710
            2011         99.10833                7.180
            2012         99.10523                7.220
            2013         99.12195                7.100
            2014         98.99389                7.270
            2015         99.17996                7.521
            2016         99.12501                8.111
            2018         99.00387                9.220
Armenia     2001         99.40016               10.912
            2011         99.74441               18.440
            2016         99.74441               17.620
            2017         99.73607               17.700
            2020         99.78861               21.206
Azerbaijan  1999         98.79180               10.900
            2007         99.59319                6.330
            2009         99.75984                5.740
            2010         99.77114                5.630
            2011         99.77863                5.420
            2012         99.77577                5.190
            2013         99.78936                4.970
            2014         99.78899                4.910

In [38]:
correlation = merged_df['literacy_rate_%'].corr(merged_df['unemployment_rate_%'])
#correlation = merged_df['gdp_%_change'].corr(merged_df['pop_%'])
print(correlation)

0.23263058185437377


In [39]:
grouped_country = merged_df.groupby('country')

for country, group_df in grouped_country:
    correlation = group_df['literacy_rate_%'].corr(group_df['unemployment_rate_%'])
    print(f"Correlation for {country}: {correlation}")

Correlation for Afghanistan: 1.0
Correlation for Albania: 0.3400196180073915
Correlation for Algeria: -0.7340360797384556
Correlation for Angola: 0.5324294728664845
Correlation for Argentina: -0.24243482393867458
Correlation for Armenia: 0.9640249301185612
Correlation for Azerbaijan: -0.990346665525529
Correlation for Bahrain: 0.9328928468419743
Correlation for Bangladesh: 0.819793667172158
Correlation for Barbados: 1.0197431391105749e-16
Correlation for Belarus: -0.733502430621145
Correlation for Belize: -0.21356580861944507
Correlation for Benin: 0.1259455407473662
Correlation for Bhutan: 0.5673363962736924
Correlation for Bolivia: 0.17700114475119386
Correlation for Bosnia and Herzegovina: 0.4143072059037442
Correlation for Botswana: 0.7236588539476246
Correlation for Brazil: 0.4896457921912556
Correlation for Brunei: 0.9300414579634252
Correlation for Bulgaria: -0.9948373622542251
Correlation for Burkina Faso: 0.9125797253377399
Correlation for Burundi: 0.5282228174975808
Correlati

C:\Users\gijsv\onedrive\documenten\ironhack\week3\project\project_1_env\lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\gijsv\onedrive\documenten\ironhack\week3\project\project_1_env\lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [40]:
missing_values = merged_df.isnull().sum()
df_clean = merged_df.dropna()

X = df_clean['literacy_rate_%']
y = df_clean['unemployment_rate_%']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     unemployment_rate_%   R-squared:                       0.054
Model:                             OLS   Adj. R-squared:                  0.053
Method:                  Least Squares   F-statistic:                     71.23
Date:                 Tue, 05 Mar 2024   Prob (F-statistic):           8.72e-17
Time:                         21:47:52   Log-Likelihood:                -3857.2
No. Observations:                 1247   AIC:                             7718.
Df Residuals:                     1245   BIC:                             7729.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.9574      